# Seventh_Test

We begin with the training test, it's important that for the moment we are not going train the model itself. That requires a GPU with almost 50 GB of VRAM.
But we can call the forward pass without problems. I wonder if we can init the model with dummy paremeters, that would be faster?
We need a way to prove that our train is efficent.

In [ ]:
from huggingface_hub import login
login()

In [ ]:
# Dependencies Installation
# Not explicitly because the repo is private.

Using Python 3.12.12 environment at: /usr
Resolved 215 packages in 2.72s
Prepared 1 package in 888ms
Uninstalled 1 package in 1ms
Installed 1 package in 2ms
 - xhuman==0.1.0 (from git+https://****@github.com/NONHUMAN-SITE/XHUMAN.git@24632fc79681baded9300ba25e06a62ea20737f8)
 + xhuman==0.1.0 (from git+https://****@github.com/NONHUMAN-SITE/XHUMAN.git@a7bce91508edcde9035edd63fcca07917201188f)


In [2]:
DS_ID = "NONHUMAN-RESEARCH/TEST_RECORD_ANNOTATIONS"
PRETRAINED_PATH = "lerobot/pi05_base"

In [3]:
from lerobot.datasets.lerobot_dataset import LeRobotDataset

dataset = LeRobotDataset(DS_ID)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

In [4]:
from xhuman.policies.pi05.configuration_pi05 import  PI05Config
device = "cuda"
policy_config = PI05Config(
        pretrained_path=PRETRAINED_PATH,  # Set pretrained path so factory loads weights
        device=device,
    )

In [5]:
from xhuman.policies.factory import make_xhuman_policy
policy = make_xhuman_policy(
        cfg=policy_config,
        ds_meta=dataset.meta,
    )

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Loading model from: lerobot/pi05_base


✓ Loaded state dict from model.safetensors
Remapped: action_in_proj.bias -> model.action_in_proj.bias
Remapped: action_in_proj.weight -> model.action_in_proj.weight
Remapped: action_out_proj.bias -> model.action_out_proj.bias
Remapped: action_out_proj.weight -> model.action_out_proj.weight
Remapped: paligemma_with_expert.gemma_expert.lm_head.weight -> model.paligemma_with_expert.gemma_expert.lm_head.weight
Remapped: paligemma_with_expert.gemma_expert.model.layers.0.input_layernorm.dense.bias -> model.paligemma_with_expert.gemma_expert.model.layers.0.input_layernorm.dense.bias
Remapped: paligemma_with_expert.gemma_expert.model.layers.0.input_layernorm.dense.weight -> model.paligemma_with_expert.gemma_expert.model.layers.0.input_layernorm.dense.weight
Remapped: paligemma_with_expert.gemma_expert.model.layers.0.mlp.down_proj.weight -> model.paligemma_with_expert.gemma_expert.model.layers.0.mlp.down_proj.weight
Remapped: paligemma_with_expert.gemma_expert.model.layers.0.mlp.gate_proj.weigh

In [6]:
from xhuman.policies.pi05.processor_pi05 import (
    make_pi05_pre_post_processors_ki,
)

preprocessor, postprocessor = make_pi05_pre_post_processors_ki(
    config=policy_config,
    dataset_stats=dataset.meta.stats,
)

In [7]:
def change_task(frame):
  # Dictionaries in Python are mutable. When 'frame' is passed to the function,
  # it's a reference to the original dictionary. Modifying it inside the function
  # will modify the original dictionary object.
  frame["task"] = "Put the fruits in the basket"
  # The function returns a reference to the same modified dictionary object.
  return frame

# --- Demonstration of the function's behavior ---
# Create a sample dictionary to observe its state.
sample_data = {"task": "Initial task", "item": "apple"}
print(f"Original dictionary before function call: {sample_data}")

# Call the function. This will modify `sample_data` in place.
# `result_data` will be a reference to the same modified dictionary object.
result_data = change_task(sample_data)

print(f"Original dictionary after function call: {sample_data}")
print(f"Returned dictionary (from function call): {result_data}")

# Verify if both variables reference the exact same object in memory.
# If True, it means the original object was modified in place.
print(f"Are 'sample_data' and 'result_data' the exact same object? {sample_data is result_data}")

Original dictionary before function call: {'task': 'Initial task', 'item': 'apple'}
Original dictionary after function call: {'task': 'Put the fruits in the basket', 'item': 'apple'}
Returned dictionary (from function call): {'task': 'Put the fruits in the basket', 'item': 'apple'}
Are 'sample_data' and 'result_data' the exact same object? True


In [9]:
import matplotlib.pyplot as plt
import numpy as np

# Get the first image tensor from the list and move it to CPU
image_tensor = img[1].squeeze(0).cpu().detach()

# Permute dimensions from (C, H, W) to (H, W, C) for matplotlib
image_np = image_tensor.permute(1, 2, 0).numpy()

# Normalize pixel values from [-3, 3] to [0, 1] for display
image_np = (image_np + 1) / 2

# Clip values to ensure they are within [0, 1] after normalization
image_np = np.clip(image_np, 0, 1)

# Plot the image
plt.imshow(image_np)
plt.title("Processed Image")
plt.axis('off') # Hide axes ticks and labels
plt.show()

NameError: name 'img' is not defined

In [13]:
batch_subtask.keys()

dict_keys(['action', 'next.reward', 'next.done', 'next.truncated', 'info', 'task', 'index', 'task_index', 'episode_index', 'frame_index', 'observation.images.left', 'observation.images.top', 'observation.images.right', 'observation.state', 'observation.language.tokens', 'observation.language.attention_mask'])

In [59]:
batch_subtask["observation.images.top"].shape

torch.Size([1, 3, 376, 672])

In [16]:
img, img_mask = policy._preprocess_images(batch_subtask)

In [17]:
tokens , mask = batch_subtask["observation.language.tokens"], batch_subtask["observation.language.attention_mask"]
# img, batch_subtask["observation.images.top"]

In [23]:
policy._detokenize_subtask(tokens)

[DEBUG] Raw subtask tokens: [     2   7071 235292   2507    573  16803    575    573  12220 235265
   4284   8277 235292 235248      0      0      0      0      0      0]
[DEBUG] Filtered tokens: [2, 7071, 235292, 2507, 573, 16803, 575, 573, 12220, 235265, 4284, 8277, 235292, 235248]
[DEBUG] Detokenized text: 'Task: put the fruits in the basket. Subtask: '


'Task: put the fruits in the basket. Subtask:'

In [30]:
mask_ai = img_mask[0]
mask_ai

tensor([True], device='cuda:0')

In [32]:
import torch

In [34]:
fal = torch.tensor([False],device="cuda")
fal

tensor([False], device='cuda:0')

In [60]:
mask_2 = [fal, mask_ai, fal]

In [40]:
img_mask

[tensor([True], device='cuda:0'),
 tensor([True], device='cuda:0'),
 tensor([True], device='cuda:0')]

In [63]:
out_put = policy.model.sample_subtask(img, mask_2, tokens, mask, 100, 1, 0.23)

In [64]:
policy._detokenize_subtask(out_put)

[DEBUG] Raw subtask tokens: [255689   3124   3351      1      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0]
[DEBUG] Filtered tokens: [255689, 3124, 3351]
[DEBUG] Detokenized text: 'Ẁ moveSub'


'Ẁ moveSub'

In [19]:
out_put

tensor([[  3351, 255331,      1,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0]], device='cuda:0')

In [30]:
from pprint import pprint
from lerobot.processor.core import TransitionKey

for i in range(1000):
    frame = change_task(dataset[i])
    time_index = frame["frame_index"]

    # Add the cached subtask to the frame BEFORE preprocessing
    if policy.cached_subtask is not None:
        frame["subtask"] = policy.cached_subtask

    if time_index % policy_config.subtask_prediction_frequency == 0:
        print("Time Index", time_index.item())
        batch_subtask = preprocessor.subtask(frame)
        print("Batch Task", batch_subtask["task"][0])
        policy.update_subtask(batch_subtask)
        print("Policy cached_subtask:", policy.cached_subtask)

        # Update frame with new subtask for next iteration
        frame["subtask"] = policy.cached_subtask

    batch = preprocessor(frame)


    if i % 50 == 0:
        print("\n" + "="*80)
        print(f"Step {i} | Time Index: {time_index}")
        print("-"*80)
        print(f"Batch Task (prompt): {batch['task'][0]}")
        print("="*80 + "\n")

Time Index 0
Batch Task Task: put the fruits in the basket. Subtask: 
Policy cached_subtask: left뀜⡯

Step 0 | Time Index: 0
--------------------------------------------------------------------------------
Batch Task (prompt): Task: left뀜⡯. State: -1 0 255 114 77 112 -1 164 -1 255 175 53 20 2 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128;
Action: 


Step 50 | Time Index: 50
--------------------------------------------------------------------------------
Batch Task (prompt): Task: left뀜⡯. State: 7 0 255 101 57 167 -1 212 0 255 164 71 47 0 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128;
Action: 

Time Index 100
Batch Task Task: put the fruits in the basket. Subtask: 
Policy cached_subtask: , the⸂

Step 100 | Time Index: 100
--------------------------------------------------------------------------------
Batch Task (prompt): Task: , the⸂. State: 7 0 255 102 60 160 -1 211 0 255 164 71 47 0 128 128 128 128 128 128 128 128 128 128 128 128 128

In [26]:
for i in range(1000):
    frame = change_task(dataset[i])
    time_index = frame["frame_index"]

    # Add the cached subtask to the frame BEFORE preprocessing
    if policy.cached_subtask is not None:
        frame["subtask"] = policy.cached_subtask


    if time_index % policy_config.subtask_prediction_frequency == 0:
        print("Time Index", time_index)
        batch_subtask = preprocessor.subtask(frame)
        print("Batch Task", batch_subtask["task"])
        policy.update_subtask(batch_subtask)
        print("Policy cached_subtask:", policy.cached_subtask)

        # Update frame with new subtask for next iteration
        frame["subtask"] = policy.cached_subtask

    batch = preprocessor(frame)
    # print("batch:", batch)
    # Now COMPLEMENTARY_DATA will contain the subtask
    complementary_data = batch.get(TransitionKey.COMPLEMENTARY_DATA, {})
    #print(complementary_data)
    if i % 50 == 0:
      print("batch_task", batch["task"])
      print("complementary_data", complementary_data)


Time Index tensor(0)
Batch Task ['Task: put the fruits in the basket. Subtask: ']
Policy cached_subtask: left뀜⡯
batch_task ['Task: left뀜⡯. State: -1 0 255 114 77 112 -1 164 -1 255 175 53 20 2 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128;\nAction: ']
complementary_data {}
batch_task ['Task: left뀜⡯. State: 7 0 255 101 57 167 -1 212 0 255 164 71 47 0 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128;\nAction: ']
complementary_data {}
Time Index tensor(100)
Batch Task ['Task: put the fruits in the basket. Subtask: ']
Policy cached_subtask: ṩ move the∐ move┱
batch_task ['Task: ṩ move the∐ move┱. State: 7 0 255 102 60 160 -1 211 0 255 164 71 47 0 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128;\nAction: ']
complementary_data {}
batch_task ['Task: ṩ move the∐ move┱. State: 73 42 184 124 151 167 42 161 75 168 163 172 5 66 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128;\nAction: ']
complementary_da

In [14]:
from lerobot.processor.core import TransitionKey

for i in range(1000):

    frame = change_task(dataset[i])
    time_index = frame["frame_index"]
    batch = preprocessor(frame)

    if i % 50 == 0:
      complementary_data = batch.get(TransitionKey.COMPLEMENTARY_DATA, {})
      print("batch_task", batch["task"])
      print("complementary_data", complementary_data)


    if time_index % policy_config.subtask_prediction_frequency == 0:
        print("Time Index", time_index)
        batch_subtask = preprocessor.subtask(frame)
        print("Batch Task", batch_subtask["task"])
        policy.update_subtask(batch_subtask)
        print(policy.cached_subtask)



batch_task ['Task: put the fruits in the basket. State: -1 0 255 114 77 112 -1 164 -1 255 175 53 20 2 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128;\nAction: ']
complementary_data {}
Time Index tensor(0)
Batch Task ['Task: put the fruits in the basket. Subtask: ']
left뀜⡯
batch_task ['Task: put the fruits in the basket. State: 7 0 255 101 57 167 -1 212 0 255 164 71 47 0 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128;\nAction: ']
complementary_data {}
batch_task ['Task: put the fruits in the basket. State: 7 0 255 102 60 160 -1 211 0 255 164 71 47 0 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128;\nAction: ']
complementary_data {}
Time Index tensor(100)
Batch Task ['Task: put the fruits in the basket. Subtask: ']
, the⸂
batch_task ['Task: put the fruits in the basket. State: 73 42 184 124 151 167 42 161 75 168 163 172 5 66 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128;\nAction: ']
complem

KeyboardInterrupt: 